## Lab. 12

### Solve the following problem using Genetic Algorithms:


Problem: Weighted N-Queen Problem


You are given an N×N chessboard, and each cell of the board has an associated weight. Your task is to find a valid placement of N queens such that the total weight of the queens is maximized, and no two queens threaten each other.





In the traditional N-Queen Problem, the goal is to place N queens on an N×N chessboard in such a way that no two queens threaten each other. In this variation, we introduce weights to the queens and aim to find a placement that maximizes the total weight of the queens while satisfying the constraint of non-threatening positions.


Constraints:

1. There should be exactly one queen in each row and each column.
2. No two queens should be placed in the same diagonal, i.e., they should not threaten each other.
3. The placement should maximize the total weight of the queens.


Representation:

Use a permutation-based representation. Each permutation represents the column position of the queen for each row. 

For example, if N=4, a valid permutation [2, 4, 1, 3] indicates that the queen in the first row is placed in column 2, the queen in the second row is placed in column 4, and so on.


Genetic Algorithm Steps:

1. *Initialization*: Generate an initial population of permutations randomly.

2. *Fitness Evaluation*: Evaluate the fitness of each permutation by calculating the total weight of the queens while considering the non-threatening positions.

3. *Selection*: Select a subset of permutations from the population based on their fitness, using selection techniques like tournament selection or roulette wheel selection.

4. *Crossover*: Perform crossover (recombination) on the selected permutations to create new offspring permutations.

5. *Mutation*: Introduce random changes (mutations) in the offspring permutations to maintain diversity in the population.

6. *Fitness Evaluation for the new individuals*: Evaluate the fitness of the new population.

7. *Form the new population*: Select the surviving individuals based on scores, with chances direct proportional with their performance.

8. Repeat steps 3-7 for a certain number of generations or until a termination condition is met (e.g., a maximum number of iterations or a satisfactory solution is found).


9. *Termination*: Return the best-performing individual (permutation) found as the solution to the problem.

Note: The fitness function used in this problem should calculate the total weight of the queens based on the positions specified by the permutation. Additionally, the fitness function should penalize solutions that violate the non-threatening constraint by assigning a lower fitness score to such permutations.

In [100]:
import random

# Generate initial population
def generate_initial_population(size, n):
    population = []
    while len(population) < size:
        permutation = random.sample(range(1, n+1), n)
        if permutation not in population:
            population.append(permutation)
    return population

size = 10
n = 4
population = generate_initial_population(size, n)
print(population)

[[1, 4, 2, 3], [3, 2, 4, 1], [4, 2, 3, 1], [2, 4, 3, 1], [1, 3, 2, 4], [4, 1, 3, 2], [3, 1, 2, 4], [2, 4, 1, 3], [4, 1, 2, 3], [4, 2, 1, 3]]


In [103]:
#Evaluate fitness
def evaluate_fitness(population, weights):
    fitness_scores = []
    for individual in population:
        n = len(individual)
        total_weight = 0
        non_threatening = True
        for i in range(n):
            for j in range(i+1, n):
                if abs(i - j) == abs(individual[i] - individual[j]):
                    total_weight = 0
                else:
                    total_weight += weights[individual[i]-1]
            fitness_scores.append(total_weight)
    return [fitness_scores[i:i+len(population)] for i in range(0, len(fitness_scores), len(population))]

weights = [2, 1, 3, 4]
fitness = evaluate_fitness(population, weights)
print(fitness)

[[6, 14, 0, 0, 6, 8, 12, 12, 0, 1], [4, 4, 3, 4, 7, 7, 0, 3, 4, 4], [12, 16, 0, 0, 9, 2, 3, 3, 3, 11], [13, 13, 4, 0, 0, 0, 12, 1, 3, 3]]


In [128]:
from random import sample

#Tournament selection
def selection(population, fitness_scores, tournament_size):
    selected = []
    for _ in range(len(population)):
        tournament = sample(population, tournament_size)
        tournament_fitness_scores = [fitness_scores[tournament.index(permutation)] for permutation in tournament]
        max_fitness = max(tournament_fitness_scores)
        max_index = tournament_fitness_scores.index(max_fitness)
        selected.append(tournament[max_index])
        
    selected = list(set(tuple(permutation) for permutation in selected))
    selected = [list(permutation) for permutation in selected]
    return selected

tournament_size = 2
selected = selection(population, fitness, tournament_size)
print(selected)

[[4, 3, 1, 2], [4, 1, 2, 3]]


In [129]:
from random import random, randint, uniform

#Crossover
def crossover(parents, crossover_rate):
    offspring = []
    for i in range(0, len(parents), 2):
        parent1 = parents[i]
        parent2 = parents[i+1]
        if random() < crossover_rate:
            n = len(parent1)
            alpha = uniform(0, 1)
            child1 = [(alpha * parent1[i] + (1 - alpha) * parent2[i]) for i in range(n)]
            child2 = [(alpha * parent2[i] + (1 - alpha) * parent1[i]) for i in range(n)]
            child1 = [int(round(value)) for value in child1]
            child2 = [int(round(value)) for value in child2]
            child1 = [min(max(value, 1), 4) for value in child1]
            child2 = [min(max(value, 1), 4) for value in child2]
            offspring.extend([child1, child2])
        else:
            offspring.extend([parent1, parent2])
    return offspring

crossover_rate = 0.8
offspring = crossover(selected, crossover_rate)
print(offspring)

[[4, 2, 1, 2], [4, 2, 2, 3]]


In [130]:
from random import random, sample

#Mutation
def mutation(offspring, mutation_rate):
    mutated_offspring = []
    for permutation in offspring:
        if random() < mutation_rate:
            index1, index2 = sample(range(len(permutation)), 2)
            permutation[index1] = min(max(permutation[index1], 1), 4)
            permutation[index2] = min(max(permutation[index2], 1), 4)
        mutated_offspring.append(permutation)
    return mutated_offspring

mutation_rate = 0.1
mutated_offspring = mutation(offspring, mutation_rate)
print(mutated_offspring)

[[4, 2, 1, 2], [4, 2, 2, 3]]


In [131]:
#Evaluate fitness of new population
new_fitness_score = evaluate_fitness(mutated_offspring, weights)
print(new_fitness_score)

[[12, 1], [0, 0], [4, 6], [0, 0]]


In [132]:
#Find best individual
def find_best_individual(population, weights):
    fitness_scores = evaluate_fitness(population, weights)
    best_index = 0
    max_fitness = max(max(scores) for scores in fitness_scores)
    for i in range(len(population)):
        if max(fitness_scores[i]) == max_fitness:
            best_index = i
            break
    return population[best_index], max_fitness

population = mutated_offspring
best_individual, best_fitness = find_best_individual(population, weights)
print(best_individual)
print(best_fitness)

[4, 2, 1, 2]
12
